# Comparison Fortran Codes APIs on GPU Environment

## Environment Modules on AIRIS

These modules must be initialized before running the jupyter-notebook:
```cpp
Currently Loaded Modulefiles:
    1) anaconda3/2023.07     
    2) openmpi/4.1.5  
    3) nvhpc/23.11
    4) llvm/12.0.0
```

In [ ]:
#!module load anaconda3/2023.07 openmpi/4.1.5 nvhpc/23.11 llvm/12.0.0

## Module List

In [1]:
!pgfortran --version


pgfortran (aka nvfortran) 23.11-0 64-bit target on x86-64 Linux -tp skylake-avx512 
PGI Compilers and Tools
Copyright (c) 2023, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.


In [2]:
!nvfortran --version


nvfortran 23.11-0 64-bit target on x86-64 Linux -tp skylake-avx512 
NVIDIA Compilers and Tools
Copyright (c) 2023, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.


In [3]:
!mpif90 --version


nvfortran 23.11-0 64-bit target on x86-64 Linux -tp skylake-avx512 
NVIDIA Compilers and Tools
Copyright (c) 2023, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.


In [4]:
!mpifort --version


nvfortran 23.11-0 64-bit target on x86-64 Linux -tp skylake-avx512 
NVIDIA Compilers and Tools
Copyright (c) 2023, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.


## `Benchmarks MPI + OpenACC + OpenMP `

### ⊗ MPI + OpenMP -> mpif90

In [5]:
%%writefile mpi_openmp.f90
program hello_mpi_openmp
  use mpi
  implicit none
  include 'omp_lib.h'

  integer :: ierr, rank, size, thread_id, num_threads, omp_status, omp_provided

  ! Initialize MPI
  call MPI_Init_thread(MPI_THREAD_FUNNELED, omp_provided, ierr)
  call MPI_Comm_rank(MPI_COMM_WORLD, rank, ierr)
  call MPI_Comm_size(MPI_COMM_WORLD, size, ierr)

  ! Print MPI information
  print *, 'Hello from MPI process ', rank, ' of ', size

  ! Parallel region with OpenMP
  !$omp parallel private(thread_id, num_threads)
  num_threads = omp_get_num_threads()
  thread_id = omp_get_thread_num()
  print *, '  Hello from thread ', thread_id, ' of ', num_threads, ' in MPI process ', rank
  !$omp end parallel

  ! Finalize MPI
  call MPI_Finalize(ierr)

end program hello_mpi_openmp

Writing mpi_openmp.f90


In [6]:
!mpif90 mpi_openmp.f90 -o mpi_openmp -fopenmp 

In [7]:
!OMP_NUM_THREADS=12 ./mpi_openmp

 Hello from MPI process             0  of             1
   Hello from thread             2  of            12  in MPI process  
            0
   Hello from thread             9  of            12  in MPI process  
            0
   Hello from thread             8  of            12  in MPI process  
            0
   Hello from thread            11  of            12  in MPI process  
            0
   Hello from thread             4  of            12  in MPI process  
            0
   Hello from thread             1  of            12  in MPI process  
            0
   Hello from thread             7  of            12  in MPI process  
            0
   Hello from thread            10  of            12  in MPI process  
            0
   Hello from thread             3  of            12  in MPI process  
            0
   Hello from thread             5  of            12  in MPI process  
            0
   Hello from thread             6  of            12  in MPI process  
            0
   Hello 

### ⊗ OpenACC + OpenMP -> mpif90

In [20]:
%%writefile openmp_openacc.f90
program main
    implicit none
    integer i, j, n, m
    integer, allocatable, dimension(:) :: task
    integer, allocatable, dimension(:) :: data

    n = 10
    allocate(task(n))
    do i=1, n
        task(i) = i * 1000
    end do
    print*, 'parallel 10 tasks, each task has 1 GPU kernel'
    
    !$omp parallel do private(i, m, data) schedule(dynamic)
    do j=1, n
        m = task(j)
        allocate(data(m))

        !$acc parallel loop copy(data) async(j)
        do i=1, m
            data(i) = j
        end do
        !$acc wait(j)

        deallocate(data)
    end do
    !$omp end parallel do

    deallocate(task)
    
end program main

Overwriting openmp_openacc.f90


In [21]:
!mpif90 openmp_openacc.f90 -o object -fopenmp -acc

In [22]:
!OMP_NUM_THREADS=8 ./object

 parallel 10 tasks, each task has 1 GPU kernel


### ⊗ MPI + OpenACC -> mpif90

In [11]:
%%writefile mpi_openacc.f90
program saxpy_acc_mpi
  use openacc
  use mpi
  implicit none

  integer :: len_global, len_local, i
  integer :: irank, nranks, igpu, ngpus, ierr, istat(MPI_STATUS_SIZE)
  real(4), allocatable, dimension(:) :: X_local, X_global, Y_global, Y_local, Y_ref
  real(4) :: a
  character(len=128) :: argv

  a = 2.0
  len_global = 1024

  ! Initialize MPI
  call MPI_Init(ierr)
  call MPI_Comm_rank(MPI_COMM_WORLD, irank, ierr)
  call MPI_Comm_size(MPI_COMM_WORLD, nranks, ierr)

  ! Check to see that the global array length is evenly divisible by the number of MPI ranks
  if (mod(len_global, nranks) .ne. 0) then
    if (irank .eq. 0) then
      write(*,'(a,5i,a,5i)'), 'The global array length, ', len_global, &
          ', is not divisible by the number of ranks, ', nranks
      call MPI_Abort(MPI_COMM_WORLD, 1, ierr)
    endif
  else
    len_local = len_global / nranks
  endif

  ! Find GPU devices and set the device number
  ngpus = acc_get_num_devices(acc_device_nvidia)
  if (ngpus .le. 0) then
    if (irank .eq. 0) then
      write(*,'(a)'), 'No NVIDIA GPUs available'
      call MPI_Abort(MPI_COMM_WORLD, 1, ierr)
    endif
  else
    igpu = mod(irank, ngpus)
    call acc_set_device_num(igpu, acc_device_nvidia)
  endif

  ! Allocate local and global arrays
  allocate(X_local(len_local))
  allocate(Y_local(len_local))
  if (irank .eq. 0) then
    allocate(X_global(len_global))
    allocate(Y_global(len_global))
    allocate(Y_ref(len_global))
  endif

  ! If root, set global and reference arrays
  if (irank .eq. 0) then
    do i = 1, len_global
      X_global(i) = i
      Y_global(i) = i + len_global
      Y_ref(i) = a * i + (i + len_global)
    enddo
  endif

  ! Scatter operands
  call MPI_Scatter( &
      X_global, &
      len_local, &
      MPI_REAL4, &
      X_local, &
      len_local, &
      MPI_REAL4, &
      0, &
      MPI_COMM_WORLD, &
      ierr &
  )
  call MPI_Scatter( &
      Y_global, &
      len_local, &
      MPI_REAL4, &
      Y_local, &
      len_local, &
      MPI_REAL4, &
      0, &
      MPI_COMM_WORLD, &
      ierr &
  )

  ! Do local calculation
!$ACC KERNELS
  do i = 1, len_local
    Y_local(i) = a*X_local(i) + Y_local(i)
  enddo
!$ACC END KERNELS

  ! Gather result
  call MPI_Gather( &
      Y_local, &
      len_local, &
      MPI_REAL4, &
      Y_global, &
      len_local, &
      MPI_REAL4, &
      0, &
      MPI_COMM_WORLD, &
      ierr &
  )

  ! Root checks result
  if (irank .eq. 0) then
    print *, 'Ran SAXPY for n = ', len_global
    if (all(Y_ref .eq. Y_global)) then
      print *, 'SUCCESS: Y_global matches Y_ref'
    else
      print *, 'FAILURE: Y_global does not match Y_ref'
      print *, 'Y_global = ', Y_global
      print *, 'Y_ref = ', Y_ref
    endif
  endif

  ! Cleanup
  deallocate(X_local)
  deallocate(Y_local)
  if (irank .eq. 0) then
    deallocate(X_global)
    deallocate(Y_global)
    deallocate(Y_ref)
  endif

  call MPI_Finalize(ierr)

end program

Writing mpi_openacc.f90


In [12]:
!mpif90 mpi_openacc.f90 -o mpi_openacc -acc 

In [13]:
!mpirun -np 2 ./mpi_openacc

 Ran SAXPY for n =          1024
 SUCCESS: Y_global matches Y_ref


### ⊗ MPI + OpenACC + OpenMP -> mpif90

In [17]:
%%writefile mpi_openacc_openmp.f90
program saxpy_acc_mpi_omp
  use openacc
  use mpi
  implicit none

  integer :: len_global, len_local, i
  integer :: irank, nranks, igpu, ngpus, ierr, istat(MPI_STATUS_SIZE)
  real(4), allocatable, dimension(:) :: X_local, X_global, Y_global, Y_local, Y_ref
  real(4), allocatable, dimension(:) :: X_local2, X_global2, Y_global2, Y_local2, Y_ref2
  real(4) :: a
  character(len=128) :: argv

  a = 2.0
  len_global = 1024

  ! Initialize MPI
  call MPI_Init(ierr)
  call MPI_Comm_rank(MPI_COMM_WORLD, irank, ierr)
  call MPI_Comm_size(MPI_COMM_WORLD, nranks, ierr)

  ! Check to see that the global array length is evenly divisible by the number of MPI ranks
  if (mod(len_global, nranks) .ne. 0) then
    if (irank .eq. 0) then
      write(*,'(a,5i,a,5i)'), 'The global array length, ', len_global, &
          ', is not divisible by the number of ranks, ', nranks
      call MPI_Abort(MPI_COMM_WORLD, 1, ierr)
    endif
  else
    len_local = len_global / nranks
  endif

  ! Find GPU devices and set the device number
  ngpus = acc_get_num_devices(acc_device_nvidia)
  if (ngpus .le. 0) then
    if (irank .eq. 0) then
      write(*,'(a)'), 'No NVIDIA GPUs available'
      call MPI_Abort(MPI_COMM_WORLD, 1, ierr)
    endif
  else
    igpu = mod(irank, ngpus)
    call acc_set_device_num(igpu, acc_device_nvidia)
  endif

  ! Allocate local and global arrays
  allocate(X_local(len_local))
  allocate(X_local2(len_local))
  allocate(Y_local(len_local))
  allocate(Y_local2(len_local))  
  if (irank .eq. 0) then
    allocate(X_global(len_global))
    allocate(Y_global(len_global))
    allocate(Y_ref(len_global))
  endif

  ! If root, set global and reference arrays
  if (irank .eq. 0) then
    do i = 1, len_global
      X_global(i) = i
      Y_global(i) = i + len_global
      Y_ref(i) = a * i + (i + len_global)
    enddo
  endif

  ! Scatter operands
  call MPI_Scatter( &
      X_global, &
      len_local, &
      MPI_REAL4, &
      X_local, &
      len_local, &
      MPI_REAL4, &
      0, &
      MPI_COMM_WORLD, &
      ierr &
  )

  call MPI_Scatter( &
      X_global, &
      len_local, &
      MPI_REAL4, &
      X_local2, &
      len_local, &
      MPI_REAL4, &
      0, &
      MPI_COMM_WORLD, &
      ierr &
  )

  call MPI_Scatter( &
      Y_global, &
      len_local, &
      MPI_REAL4, &
      Y_local, &
      len_local, &
      MPI_REAL4, &
      0, &
      MPI_COMM_WORLD, &
      ierr &
  )
  
  call MPI_Scatter( &
      Y_global, &
      len_local, &
      MPI_REAL4, &
      Y_local2, &
      len_local, &
      MPI_REAL4, &
      0, &
      MPI_COMM_WORLD, &
      ierr &
  )

  ! Do local calculation
  !$ACC KERNELS
  do i = 1, len_local
    Y_local(i) = a*X_local(i) + Y_local(i)
  enddo
  !$ACC END KERNELS

  !$omp parallel do private(i) schedule(dynamic)
  do i = 1, len_local
    Y_local2(i) = a*X_local2(i) + Y_local2(i)
  enddo
  !$omp end parallel do  
    
    
  ! Gather result
  call MPI_Gather( &
      Y_local, &
      len_local, &
      MPI_REAL4, &
      Y_global, &
      len_local, &
      MPI_REAL4, &
      0, &
      MPI_COMM_WORLD, &
      ierr &
  )

  ! Root checks result
  if (irank .eq. 0) then
    print *, 'Ran SAXPY for n = ', len_global
    if (all(Y_ref .eq. Y_global)) then
      print *, 'SUCCESS: Y_global matches Y_ref'
    else
      print *, 'FAILURE: Y_global does not match Y_ref'
      print *, 'Y_global = ', Y_global
      print *, 'Y_ref = ', Y_ref
    endif
  endif

  ! Cleanup
  deallocate(X_local)
  deallocate(X_local2)
  deallocate(Y_local)
  deallocate(Y_local2)  
  if (irank .eq. 0) then
    deallocate(X_global)
    deallocate(Y_global)
    deallocate(Y_ref)
  endif

  call MPI_Finalize(ierr)

end program

Overwriting mpi_openacc_openmp.f90


In [18]:
!mpif90 mpi_openacc_openmp.f90 -o mpi_openacc_openmp -fopenmp -acc

In [19]:
!OMP_NUM_THREADS=4 mpirun -np 2 ./mpi_openacc_openmp

 Ran SAXPY for n =          1024
 SUCCESS: Y_global matches Y_ref


## Limpando os arquivos remanescentes

In [23]:
!rm -rf mm* file* fspeed* mpi_openmp* object* hello_* mpi_openacc* openmp_*